#Recommendation Systems in Machine Learning

## Week 3 - Content Based Recommendation Systems and Word Embeddings

This week, we will build a content-based recommendation system on the MovieLens dataset using tf-idf and cosine similarity. The system will make recommendations which are similar in content to a given query, which can be stated preferences or a sample movie. The main description of content in the MovieLens dataset is the genres label. For now, we will work with these discrete labels rather than natural language documents; but even in this simplified setting, you will see how tf-idf can make better recommendations than a naive bag of words approach. 

We have written most of the skeleton code for you. Your task is to implement two functions: tfidf, where you compute the tf-idf feature vector for each movie, and compute_pairwise_cosine_similarities, where you implement its namesake functionality. 



We begin by mounting our drive and loading the dataset as we did in week 1.

In [4]:
from google.colab import drive
drive.mount('/content/drive/')
DRIVE_PREFIX = '/content/drive/MyDrive/Colab Notebooks'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [7]:
import zipfile as zf
DRIVE_PREFIX = '/content/drive/MyDrive/Colab Notebooks/'
files = zf.ZipFile(DRIVE_PREFIX + "ml-latest-small.zip", 'r')
files.extractall("MovieLens-Data")
files.close()

In [8]:
import pandas as pd
movies = pd.read_csv("MovieLens-Data/ml-latest-small/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Here we go over the dataset to build a vocabulary of genres. This means forming correspondences between genre and index, so that we can easily determine which entry in the feature vector represents a term, or the other way around. 

In [10]:
def build_vocabulary(movies_df):

  """Create an index of genres
  
  Returns:
    genre_to_idx: a dict that maps from genre name to index.
    idx_to_genre: an ordered list of genre names.
    genre_document_frequency: a dict that maps from genre name to
      its document frequency. used later for computing idf.
  """

  genre_to_idx = {}
  genre_document_frequency = {}
  idx_to_genre = []
  for _, movie in movies_df.iterrows():
    genres = movie['genres']
    for genre in genres.split('|'):
      if genre not in idx_to_genre:
        genre_to_idx[genre] = len(idx_to_genre)
        genre_document_frequency[genre] = 0
        idx_to_genre.append(genre)
      genre_document_frequency[genre] += 1
  return genre_to_idx, idx_to_genre, genre_document_frequency

genre_to_idx, idx_to_genre, doc_freqs = build_vocabulary(movies)

We have implemented the bag-of-words feature for you, which you can reference when implementing tf-idf. For this dataset, since the genres list never repeats terms, its bag of words feature is equivalent to the binary representation we discussed in lecture. If we were working with text documents, bag-of-words will most likely contain integer entries (not just 0 and 1).

In [18]:
import numpy as np

def bag_of_words(genres_string):
  """Create a bag-of-words feature vector for a genres string

  Args:
    genres_string: the 'genres' column of a movie. for example, 
      'Adventure|Animation|Children|Comedy|Fantasy'
  
  Returns:
    a one-dimensional np array
  """

  genres_list = genres_string.split('|')
  bow_vector = np.zeros(len(idx_to_genre))
  for genre in genres_list:
    bow_vector[genre_to_idx[genre]] += 1
  return bow_vector

In [38]:
bag_of_words('Adventure|Romance|Fantasy')

array([1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

## TF-IDF

Next, you will implement a function that creates the tf-idf feature vector for movies. The definition of tf-idf is:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/10109d0e60cc9d50a1ea2f189bac0ac29a030a00)

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/dd4f8a91dd0d28a11c00c94a13a315a5b49a8070)

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/ac67bc0f76b5b8e31e842d6b7d28f8949dab7937)

Feel free to declare global variables outside the function body.


In [39]:
def tfidf(genres_string):
  """Create a tf-idf feature vector for a genres string

  Args:
    genres_string: the 'genres' column of a movie. for example, 
      'Adventure|Animation|Children|Comedy|Fantasy'
  
  Returns:
    a one-dimensional np array
  """
  b_o_w = bag_of_words(genres_string)
  tf = b_o_w
  
  N = len(movies)
  denom = np.zeros(len(idx_to_genre))
  for genre, df in doc_freqs.items():
    denom[genre_to_idx[genre]] = df

  idf = np.log(N / denom)

  return tf * idf

You should see that the output is not a binary vector. Because tf-idf favors terms that are rarer across documents, these genres are given higher weights. The difference in weight will come into play when we compute cosine similarities later, where the metric will prioritize matching a higher weighted genre over a lower weighted one. 

In [40]:
tfidf('Adventure|Romance|Fantasy')

array([2.04295659, 0.        , 0.        , 0.        , 2.52619067,
       1.80894594, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

Here we compute tf-idf vectors for all the movies and compile them into one data structure. There are many ways to do this, for example, you might add a column to the movies dataframe to store the tf-idf vectors. However, to make use of numpy optimizations later, we choose to stack the vectors into one large numpy array.

In [41]:
tfidf_vector_list = [tfidf(movie['genres']) for _, movie in movies.iterrows()]
tfidf_array = np.stack(tfidf_vector_list, axis=0)

tfidf_array.shape

(9742, 20)

## Cosine Similarity

In the next part, we ask you to implement a function that computes the pairwise cosine similarities between movies' tfidf features. The naive way to do this is with nested for loops that simply go over every possible pair, but we recommend using matrix multiplication and other numpy operations to take advantage of its speed optimizations. Ideally, you can do this without any for loops. 

We implemented the pairwise dot product function as example of how to utilize matrix multiplication. Few things to note:


```
A @ B    # shorthand for matrix multiplication of A and B
A.T      # A transpose
```

Remember that the formula of cosine similarity is

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/1d94e5903f7936d3c131e040ef2c51b473dd071d)

The numerator is just the dot product, so your main problem is to figure out how to compute the pairwise magnitude products on the denominator, ideally without looping over every pair.

In [62]:
def compute_pairwise_dot_products(feature_array):
  '''Compute pairwise dot products

  Args:
    feature_array: an array of feature vectors, computed
      like tfidf_array above. 

  Returns:
    a N by N matrix where entry [i, j] is the dot product
    between the ith and jth feature vectors.
  '''

  pairwise_dot_products = feature_array @ feature_array.T
  return pairwise_dot_products

def compute_pairwise_cosine_similarities(feature_array):
  '''Compute pairwise cosine similarities

  Args:
    feature_array: an array of feature vectors, computed
      like tfidf_array above. 

  Returns:
    a N by N matrix where entry [i, j] is the cosine similarity
    between the ith and jth feature vectors.
  '''
  magnitudes = np.sqrt(np.sum(feature_array * feature_array, axis=1))
  mag_products = np.outer(magnitudes, magnitudes)
  return compute_pairwise_dot_products(feature_array) / mag_products 

cos_sims = compute_pairwise_cosine_similarities(tfidf_array)

This function retrieves the top k movies most similar to its input.

In [63]:
def recommend(movie_name, top_k):
  movie_idx = movies.index[movies['title'] == movie_name][0]
  rec_idxs = np.argsort(-cos_sims[movie_idx])[:top_k]
  return movies.iloc[rec_idxs]

Let's make some recommendations!

You will see that the top recommendations have exactly the same genres as our input. This is because we are only relying on the genres label to make recommendations. One improvement is to also tf-idf-ize the movie names, which will allow the system to pick up movies from the same franchise, as they often share a prefix like 'Star Wars: xxx'.

In [64]:
recommend('Toy Story (1995)', 20)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3000,4016,"Emperor's New Groove, The (2000)",Adventure|Animation|Children|Comedy|Fantasy
8219,103755,Turbo (2013),Adventure|Animation|Children|Comedy|Fantasy
6194,45074,"Wild, The (2006)",Adventure|Animation|Children|Comedy|Fantasy
1706,2294,Antz (1998),Adventure|Animation|Children|Comedy|Fantasy
6948,65577,"Tale of Despereaux, The (2008)",Adventure|Animation|Children|Comedy|Fantasy
2809,3754,"Adventures of Rocky and Bullwinkle, The (2000)",Adventure|Animation|Children|Comedy|Fantasy
6486,53121,Shrek the Third (2007),Adventure|Animation|Children|Comedy|Fantasy
7760,91355,Asterix and the Vikings (Astérix et les Viking...,Adventure|Animation|Children|Comedy|Fantasy
8927,136016,The Good Dinosaur (2015),Adventure|Animation|Children|Comedy|Fantasy
